# functional APIでKerasを始めてみよう

functional APIは，複数の出力があるモデルや有向非巡回グラフ，共有レイヤーを持ったモデルなどの複雑なモデルを定義するためのインターフェースです．

ここでは`Sequential`モデルについて既に知識があることを前提として説明します．

シンプルな例から見てきましょう．

## 例1: 全結合ネットワーク

下記のネットワークは`Sequential`モデルによっても定義可能ですが，
functional APIを使ったシンプルな例を見てきましょう．

- レイヤーのインスタンスは関数呼び出し可能で，戻り値としてテンソルを返します
- `Model`を定義することで入力と出力のテンソルは接続されます
- 上記で定義したモデルは`Sequential`と同様に利用可能です

In [4]:
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ダミーデータの作成
import numpy as np
data = np.random.random((1000, 784))
labels = np.random.randint(2, size=(1000, 10))

model.fit(data, labels)  # starts training

Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 11.9001 - acc: 0.1270


## 全てのモデルはレイヤーと同じように関数呼び出し可能です

functional APIを利用することで，訓練済みモデルの再利用が簡単になります：全てのモデルを，テンソルを引数としたlayerのように扱うことができます．これにより，モデル構造だけでなく，モデルの重みも再利用できます．

In [0]:
x = Input(shape=(784,))
# This works, and returns the 10-way softmax we defined above.
y = model(x)

一連のシーケンスを処理するモデルを簡単に設計できます．
例えば画像識別モデルをたった1行で動画識別モデルに応用できます．

In [0]:
from keras.layers import TimeDistributed

# Input tensor for sequences of 20 timesteps,
# each containing a 784-dimensional vector
input_sequences = Input(shape=(20, 784))

# This applies our previous model to every timestep in the input sequences.
# the output of the previous model was a 10-way softmax,
# so the output of the layer below will be a sequence of 20 vectors of size 10.
processed_sequences = TimeDistributed(model)(input_sequences)

## 多入力多出力モデル

functional APIは複数の入出力を持ったモデルに最適です．
複数の複雑なデータストリームを簡単に扱うことができます．

Twitterの新しいニュースヘッドラインを受信した際，そのツイートのリツイートやライクの回数を予測する例を考えます．主な入力はヘッドラインの単語のシーケンスですが，スパイスとして，ヘッドラインの投稿時間などのデータを入力として追加します．
このモデルは2つの損失関数によって訓練されます．モデルにおける初期の主損失関数を使うことは，深い層を持つモデルにとっては良い正則化の構造です．

以下がモデルの図になります．

<img src="https://s3.amazonaws.com/keras.io/img/multi-input-multi-output-graph.png" alt="multi-input-multi-output-graph" style="width: 400px;"/>

functional APIを利用してこのネットワークを実装してみましょう．

main inputはヘッドラインを整数のシーケンス（それぞれの整数は単語をエンコードしたしたもの）として受け取ります．
整数の範囲は1から10000となり（単語数は10000語），各シーケンスは長さ100単語で構成されます．

In [0]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# A LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
lstm_out = LSTM(32)(x)

ここでは補助損失を追加し，LSTMとEmbeddedingレイヤーをスムーズに訓練できるようにしますが，モデルでは主損失がはるかに高くなります．

In [0]:
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

この時点で，auxiliary_inputをLSTM出力と連結してモデルに入力します．

In [0]:
import keras
auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

2つの入力と2つの出力を持ったモデルを定義します．

In [0]:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

モデルをコンパイルし，補助損失に0.2の重み付けを行います．
様々な`loss_weights`や`loss`を対応付けるためにリストもしくは辞書を利用します．
`loss`に1つの損失関数を与えた場合，全ての出力に対して同一の損失関数が適用されます．

In [0]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              loss_weights=[1., 0.2])

モデルに入力と教師データをリストで渡すことで訓練できます．


`model.fit([headline_data, additional_data], [labels, labels],
          epochs=50, batch_size=32)`


入力と出力に名前付けを行っていれば（"name"引数を利用），下記のような方法でモデルをコンパイルできます．


```
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

# And trained it via:
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          epochs=50, batch_size=32)
```


ValueError: ignored